# Examples of utils provided by ReGraph's neo4j module

In [1]:
import networkx as nx

from regraph import Rule, plot_rule
from regraph.neo4j.graphs import Neo4jGraph
from regraph.neo4j.cypher_utils import *

## Initializing Neo4j graph

0. When installing neo4j you will be asked to choose login/password for you dbs (here its "neo4j"/"admin"), if you choose other -- change in the cell below.
1. To start neo4j server run `sudo service neo4j start`
2. Check status by running `sudo service neo4j status`. Here you can check the _bolt_ port, change the cell below if different from 7687
3. You can query the db by using the neo4j browser, the address can be found also in the result of 'status', e.g. "Remote interface available at http://localhost:7474/".

In [2]:
# initialize the neo4j driver, wrappped into Neo4jGraph object
g = Neo4jGraph(uri="bolt://localhost:7687", user="neo4j", password="admin")

In [3]:
# here we clear the db
res = g.clear()

In [4]:
nodes = [
    ("a", {"name": "Jack", "age": 23, "hobby": {"hiking", "music"}, "weight": 75}), 
    ("b", {"name": "Bob", "age": 24, "hobby": {"sport", "music"}, "height": 178}),
    "c", 
    ("d", {"name": "Paul"}), "e", "f"
]
edges = [
    ("a", "b", {"type": {"friends", "colleagues"}}), 
    ("d", "b", {"type": "enemies"}), 
    ("a", "c"),
    ("d", "a", {"type": "friends"}),
    ("e", "a"), 
    ("f", "d")
]
g.add_nodes_from(nodes)
g.add_edges_from(edges)

## Primitive operations on Neo4j graph

### Basic operations

In [5]:
g.add_node("x", {"name": "Roberto"})

In [6]:
g.add_edge("x", "c", {"type": {"friends", "colleagues"}, "since": 1993})

In [ ]:
g.nodes()

['a', 'b', 'c', 'd', 'e', 'f', 'x']

In [ ]:
g.edges()

[('e', 'a'),
 ('d', 'a'),
 ('a', 'b'),
 ('d', 'b'),
 ('x', 'c'),
 ('a', 'c'),
 ('f', 'd')]

In [ ]:
g.get_node('a')

In [ ]:
g.get_edge('a', 'b')

In [ ]:
g.merge_nodes1(["a", "b"])

In [12]:
g.merge_nodes1(["a_b", "d"])

MATCH (a_b:node { id : 'a_b'}), (d:node { id : 'd'}) 
// accumulate all the attrs of the nodes to be merged
WITH [] as new_props, a_b, d
WITH new_props + REDUCE(pairs = [], k in keys(a_b) | 
	pairs + REDUCE(inner_pairs = [], v in a_b[k] | 
		inner_pairs + {key: k, value: v})) as new_props, a_b, d
WITH new_props + REDUCE(pairs = [], k in keys(d) | 
	pairs + REDUCE(inner_pairs = [], v in d[k] | 
		inner_pairs + {key: k, value: v})) as new_props, a_b, d
WITH apoc.map.groupByMulti(new_props, 'key') as new_props, a_b, d
WITH apoc.map.fromValues(REDUCE(pairs=[], k in keys(new_props) | 
	pairs + [k, REDUCE(values=[], v in new_props[k] | 
		values + CASE WHEN v.value IN values THEN [] ELSE v.value END)])) as new_props, a_b, d
SET a_b = new_props
WITH a_b as merged_node, d
// search for a node with the same id as the clone id
OPTIONAL MATCH (same_id_node:node { id : 'a_b_d'}) 
WITH same_id_node,  CASE WHEN same_id_node IS NOT NULL THEN (coalesce(same_id_node.count, 0) + 1) ELSE 0 END AS same_id

'a_b_d'

### Cloning

By default if we clone a node without specifying the new id for it, the id is automatically derived from the original one (however it takes some additional time to generate it, and some naming collisions may happen in the future when using clone operation inside of complex queries)

In [ ]:
clone1_res = g.clone_node('a')
print("Created clone by the name: ", clone1_res)
print("Properties of the resulting node: ")
print(g.get_node(clone1_res))
print("Properties of incident edges are also cloned, e.g: ")
print(g.get_edge(clone1_res, 'b'))

We can specify the parameter `ignore_naming=True` to avoid automatic id derivation. In this case cloning uses Neo4j native ids (this mode is more suitable for using clone operation inside of complex queries as the Neo4j id have the guaranteed uniqueness)

In [ ]:
print("Here we ignore pretty naming of new clones\n")
clone2_res = g.clone_node('a', ignore_naming=True)
print("Created clone by the name: ", clone2_res)
print("Properties of the resulting node: ")
print(g.get_node(clone2_res))
print("Properties of incident edges are also cloned, e.g: ")
print(g.get_edge(clone2_res, 'b'))

The third option is to specify the id of the new node manually

In [ ]:
clone3_res = g.clone_node('a', 'a_clone')
print("Created clone by the name: ", clone3_res)
print("Properties of the resulting node: ")
print(g.get_node(clone3_res))
print("Properties of incident edges are also cloned, e.g: ")
print(g.get_edge(clone3_res, 'b'))

In [ ]:
merge1_res = g.merge_nodes(["a", "b"])
print("Properties of the resulting node: {}".format(merge1_res))
print(g.get_node(merge1_res))

In [ ]:
merge2_res = g.merge_nodes(["a_b", "d"])
print("Properties of the resulting node: {}".format(merge2_res))
print(g.get_node(merge2_res))

## Rewriting Neo4j graph

### 1. Create an SqPO rewriting rule

In [ ]:
pattern = nx.DiGraph()
pattern.add_nodes_from(["x", "y", "z"])
pattern.add_edges_from([("y", "x"), ("y", "z"), ("z", "z")])

In [ ]:
rule = Rule.from_transform(pattern)
clone_name1, _ = rule.inject_clone_node("y")
clone_name2, _ = rule.inject_clone_node("y")
rule.inject_remove_edge(clone_name2, "z")
rule.inject_remove_node("x")
rule.inject_add_node("new_node")
rule.inject_add_edge("new_node", "z")

In [ ]:
plot_rule(rule)

In [ ]:
instances = g.find_matching(pattern)
print("Instances: ", instances)

In [ ]:
rhs_g = g.rewrite(rule, instances[0])
print("\n\nRewriting rule to commands: \n")
print(rule.to_commands())

In [ ]:
rhs_g